In [1]:
import pandas as pd
train = pd.read_csv("train.tsv",sep="\t",index_col=0)
test = pd.read_csv("test.tsv",sep="\t",index_col=0)
sample_submit = pd.read_csv("sample_submit.csv",sep="\t",index_col=0,header=None)

In [6]:
train.head()
#class = 0 左
#class = 1 平衡
#class = 2 右

,left_weight,left_distance,right_weight,right_distance
0,4,5,4,3
1,3,2,1,5
3,1,4,5,1
4,5,4,4,4
5,2,2,4,1


欠陥値やカテゴリ変数有無、外れ値がないか確認

In [4]:
train.describe()

,class,left_weight,left_distance,right_weight,right_distance
count,312.000000,312.000000,312.000000,312.000000,312.000000
mean,1.009615,3.025641,2.942308,2.987179,3.003205
std,0.957239,1.436541,1.444539,1.400446,1.444578
min,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,2.000000,2.000000,2.000000,2.000000
50%,1.000000,3.000000,3.000000,3.000000,3.000000
75%,2.000000,4.000000,4.000000,4.000000,4.000000
max,2.000000,5.000000,5.000000,5.000000,5.000000


目的変数に対しての各カラムの相関係数の確認

→特徴量が大きい列を学習に使う。その際、多重共線性にも注意


多重共線性とは、説明変数同士に相関があること。そのまま学習に使うと二重で学習されてしまう。

In [7]:
train.corrwith(train['class'])
#相関係数の解釈
#classを減らす要因としてleft_XXが関係している
#classを増やす要因としてright_XXが関係している
# 重さと距離については左右で係数が異なっているが現実世界では同じなはず


class             1.000000
left_weight      -0.374309
left_distance    -0.401885
right_weight      0.415045
right_distance    0.406904
dtype: float64

In [4]:
import lightgbm as lgb
model = lgb.LGBMClassifier(boosting_type='goss', max_depth=5, random_state=0)

In [5]:
y_train = train['class']
y_test = test['class']
X_train = train.drop(['class'])
X_test = test.drop(['class'])

eval_set = [(X_test, y_test)]
callbacks = []
callbacks.append(lgb.early_stopping(stopping_rounds=10))
callbacks.append(lgb.log_evaluation())
model.fit(X_train, y_train, eval_set=eval_set, callbacks=callbacks)

KeyError: 'class'